In [2]:
import pandas as pd
#path = 'sms.tsv'

sms = pd.read_table('spam.tsv', header=None, names=['label', 'message'], encoding='latin-1')


In [3]:
sms.head(10)


,label,message
0,"v1,v2,,,",NaN
1,"ham,""Go until jurong point, crazy.. Available ...",NaN
2,"ham,Ok lar... Joking wif u oni...,,,",NaN
3,"spam,Free entry in 2 a wkly comp to win FA Cup...",NaN
4,"ham,U dun say so early hor... U c already then...",NaN
5,"ham,""Nah I don't think he goes to usf, he live...",NaN
6,"spam,""FreeMsg Hey there darling it's been 3 we...",NaN
7,"ham,Even my brother is not like to speak with ...",NaN
8,"ham,As per your request 'Melle Melle (Oru Minn...",NaN
9,"spam,WINNER!! As a valued network customer you...",NaN


In [5]:
sms.label.value_counts()
sms[label]


NameError: name 'label' is not defined

In [4]:
sms['label_num'] = sms.label.map({'ham':0, 'spam':1})



In [5]:
sms.head(10)


,label,message,label_num
0,"v1,v2,,,",NaN,NaN
1,"ham,""Go until jurong point, crazy.. Available ...",NaN,NaN
2,"ham,Ok lar... Joking wif u oni...,,,",NaN,NaN
3,"spam,Free entry in 2 a wkly comp to win FA Cup...",NaN,NaN
4,"ham,U dun say so early hor... U c already then...",NaN,NaN
5,"ham,""Nah I don't think he goes to usf, he live...",NaN,NaN
6,"spam,""FreeMsg Hey there darling it's been 3 we...",NaN,NaN
7,"ham,Even my brother is not like to speak with ...",NaN,NaN
8,"ham,As per your request 'Melle Melle (Oru Minn...",NaN,NaN
9,"spam,WINNER!! As a valued network customer you...",NaN,NaN


In [6]:
X = sms.message
y = sms.label_num
print(X.shape)
print(y.shape)


(5575,)
(5575,)


In [7]:
import sklearn
from sklearn.cross_validation import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=1)
print(X_train.shape)
print(X_test.shape)
print(y_train.shape)
print(y_test.shape)


ImportError: No module named 'sklearn.cross_validation'

In [ ]:
from sklearn.feature_extraction.text import CountVectorizer
#vect = CountVectorizer()

vect = CountVectorizer()


# learn training data vocabulary, then use it to create a document-term matrix
vect.fit(X_train)
X_train_dtm = vect.transform(X_train)


# equivalently: combine fit and transform into a single step
X_train_dtm = vect.fit_transform(X_train)


# examine the document-term matrix
X_train_dtm

<4179x7456 sparse matrix of type '<class 'numpy.int64'>'
	with 55209 stored elements in Compressed Sparse Row format>

In [ ]:
X_test_dtm = vect.transform(X_test)
X_test_dtm


<1393x7456 sparse matrix of type '<class 'numpy.int64'>'
	with 17604 stored elements in Compressed Sparse Row format>

In [ ]:
from sklearn.naive_bayes import MultinomialNB
nb = MultinomialNB()


# train the model using X_train_dtm
nb.fit(X_train_dtm, y_train)


# make class predictions for X_test_dtm
y_pred_class = nb.predict(X_test_dtm)



In [ ]:
from sklearn import metrics
metrics.accuracy_score(y_test, y_pred_class)


0.9885139985642498

In [ ]:
metrics.confusion_matrix(y_test, y_pred_class)



array([[1203,    5],
       [  11,  174]])

In [ ]:
X_test[3132]



"LookAtMe!: Thanks for your purchase of a video clip from LookAtMe!, you've been charged 35p. Think you can do better? Why not send a video in a MMSto 32323."

In [ ]:
y_pred_prob = nb.predict_proba(X_test_dtm)[:, 1]
y_pred_prob



array([2.87744864e-03, 1.83488846e-05, 2.07301295e-03, ...,
       1.09026171e-06, 1.00000000e+00, 3.98279868e-09])

In [ ]:
metrics.roc_auc_score(y_test, y_pred_prob)


0.9866431000536962

In [ ]:
from sklearn.metrics import classification_report
print(classification_report(y_test, y_pred_class))
target_names=['ham','spam']
print(classification_report(y_test, y_pred_class, target_names=target_names))

             precision    recall  f1-score   support

          0       0.99      1.00      0.99      1208
          1       0.97      0.94      0.96       185

avg / total       0.99      0.99      0.99      1393

             precision    recall  f1-score   support

        ham       0.99      1.00      0.99      1208
       spam       0.97      0.94      0.96       185

avg / total       0.99      0.99      0.99      1393



In [ ]:
from sklearn.linear_model import LogisticRegression
logreg = LogisticRegression()


# train the model using X_train_dtm
logreg.fit(X_train_dtm, y_train)


# make class predictions for X_test_dtm
y_pred_class = logreg.predict(X_test_dtm)


# calculate predicted probabilities for X_test_dtm (well calibrated)
y_pred_prob = logreg.predict_proba(X_test_dtm)[:, 1]
y_pred_prob


array([0.01269556, 0.00347183, 0.00616517, ..., 0.03354907, 0.99725053,
       0.00157706])

In [ ]:
metrics.accuracy_score(y_test, y_pred_class)

0.9877961234745154

In [ ]:
metrics.roc_auc_score(y_test, y_pred_prob)


0.9936817612314301

In [ ]:
X_train_tokens = vect.get_feature_names()
len(X_train_tokens)


7456

In [ ]:
target_names=['ham','spam']
print(classification_report(y_test, y_pred_class, target_names=target_names))

             precision    recall  f1-score   support

        ham       0.99      1.00      0.99      1208
       spam       0.99      0.91      0.95       185

avg / total       0.99      0.99      0.99      1393



In [ ]:
# Naive Bayes counts the number of times each token appears in each class
nb.feature_count_


array([[ 0.,  0.,  0., ...,  1.,  1.,  1.],
       [ 5., 23.,  2., ...,  0.,  0.,  0.]])

In [ ]:
# rows represent classes, columns represent tokens
nb.feature_count_.shape


(2, 7456)

In [ ]:
# number of times each token appears across all HAM messages
ham_token_count = nb.feature_count_[0, :]
ham_token_count



array([0., 0., 0., ..., 1., 1., 1.])

In [ ]:
# number of times each token appears across all SPAM messages
spam_token_count = nb.feature_count_[1, :]
spam_token_count

array([ 5., 23.,  2., ...,  0.,  0.,  0.])

In [ ]:
# create a DataFrame of tokens with their separate ham and spam counts
tokens = pd.DataFrame({'token':X_train_tokens, 'ham':ham_token_count, 'spam':spam_token_count}).set_index('token')
tokens.head()


,ham,spam
token,,
00,0.0,5.0
000,0.0,23.0
008704050406,0.0,2.0
0121,0.0,1.0
01223585236,0.0,1.0


In [ ]:
# examine 5 random DataFrame rows
tokens.sample(5, random_state=6)


,ham,spam
token,,
very,64.0,2.0
nasty,1.0,1.0
villa,0.0,1.0
beloved,1.0,0.0
textoperator,0.0,2.0


In [ ]:

# Naive Bayes counts the number of observations in each class
nb.class_count_


array([3617.,  562.])

In [ ]:
# add 1 to ham and spam counts to avoid dividing by 0
tokens['ham'] = tokens.ham + 1
tokens['spam'] = tokens.spam + 1
tokens.sample(5, random_state=6)


,ham,spam
token,,
very,65.0,3.0
nasty,2.0,2.0
villa,1.0,2.0
beloved,2.0,1.0
textoperator,1.0,3.0


In [ ]:
tokens['ham'] = tokens.ham / nb.class_count_[0]
tokens['spam'] = tokens.spam / nb.class_count_[1]
tokens.sample(5, random_state=6)

,ham,spam
token,,
very,0.017971,0.005338
nasty,0.000553,0.003559
villa,0.000276,0.003559
beloved,0.000553,0.001779
textoperator,0.000276,0.005338


In [ ]:
tokens['spam_ratio'] = tokens.spam / tokens.ham
tokens.sample(5, random_state=6)

,ham,spam,spam_ratio
token,,,
very,0.017971,0.005338,0.297044
nasty,0.000553,0.003559,6.435943
villa,0.000276,0.003559,12.871886
beloved,0.000553,0.001779,3.217972
textoperator,0.000276,0.005338,19.307829


In [ ]:
tokens.sort_values('spam_ratio', ascending=False)



,ham,spam,spam_ratio
token,,,
claim,0.000276,0.158363,572.798932
prize,0.000276,0.135231,489.131673
150p,0.000276,0.087189,315.361210
tone,0.000276,0.085409,308.925267
guaranteed,0.000276,0.076512,276.745552
18,0.000276,0.069395,251.001779
cs,0.000276,0.065836,238.129893
www,0.000553,0.129893,234.911922
1000,0.000276,0.056940,205.950178


In [ ]:
tokens.loc['dating', 'spam_ratio']


83.66725978647686

In [ ]:
# remove English stop words
vect = CountVectorizer(stop_words='english')


In [ ]:
from sklearn.neighbors import KNeighborsClassifier

# instantiate the model (with the default parameters)
knn = KNeighborsClassifier()
knn.fit(X_train_dtm, y_train)

KNeighborsClassifier(algorithm='auto', leaf_size=30, metric='minkowski',
           metric_params=None, n_jobs=1, n_neighbors=5, p=2,
           weights='uniform')

In [ ]:
#knn.predict()
y_pred_class = knn.predict(X_test_dtm)

In [ ]:
metrics.accuracy_score(y_test, y_pred_class)


0.91816223977028

In [ ]:
metrics.confusion_matrix(y_test, y_pred_class)

array([[1208,    0],
       [ 114,   71]])

In [ ]:
from sklearn.metrics import classification_report
print(classification_report(y_test, y_pred_class))
target_names=['ham','spam']
print(classification_report(y_test, y_pred_class, target_names=target_names))

             precision    recall  f1-score   support

          0       0.91      1.00      0.95      1208
          1       1.00      0.38      0.55       185

avg / total       0.93      0.92      0.90      1393

             precision    recall  f1-score   support

        ham       0.91      1.00      0.95      1208
       spam       1.00      0.38      0.55       185

avg / total       0.93      0.92      0.90      1393



In [ ]:
#y_pred_class = nb.predict(X_test_dtm)
X_test_dtm[0]

<1x7456 sparse matrix of type '<class 'numpy.int64'>'
	with 4 stored elements in Compressed Sparse Row format>

In [ ]:
y_pred_class = nb.predict(X_test_dtm)
X_test[:10]



1078                         Yep, by the pretty sculpture
4028        Yes, princess. Are you going to make me moan?
958                            Welp apparently he retired
4642                                              Havent.
4674    I forgot 2 ask ü all smth.. There's a card on ...
5461    Ok i thk i got it. Then u wan me 2 come now or...
4210    I want kfc its Tuesday. Only buy 2 meals ONLY ...
4216                           No dear i was sleeping :-P
1603                            Ok pa. Nothing problem:-)
1504                      Ill be there on  &lt;#&gt;  ok.
Name: message, dtype: object

In [ ]:
#print(X_test_dtm.toarray
print("how many number of message u want check?")
n=int(input())
while(n!=0):
    print("message number?")
    number=int(input())
    if y_pred_class[number]==0:
        print('ham')
        print(X_test.iloc[number])
    else:
        print('spam')
        print(X_test.iloc[number])
    n-=1
#print(y_pred_class[0])


how many number of message u want check?
3
message number?
0
ham
Yep, by the pretty sculpture
message number?
4
ham
I forgot 2 ask ü all smth.. There's a card on da present lei... How? Ü all want 2 write smth or sign on it?
message number?
5
ham
Ok i thk i got it. Then u wan me 2 come now or wat?


In [ ]:
y_pred_class=nb.predict(X_test_dtm[0:10])
